In [1]:
# libraries

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast
from sklearn.model_selection import train_test_split


2025-01-26 15:29:20.376450: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df1 = pd.read_csv("data/large_board_dataset.csv")
df2 = pd.read_csv("data/large_board_dataset2.csv")
df3 = pd.read_csv("data/large_board_dataset3.csv")
df4 = pd.read_csv("data/player_one_moves.csv")
df5 = pd.read_csv("data/player_two_moves.csv")


df = pd.concat([df1, df2, df3, df4, df5], axis =0)


In [4]:
df.head()
df["x"] = df["x"].apply(lambda x: np.array(ast.literal_eval(x)).reshape(2, 6, 7))

In [5]:
x = df["x"]
y = df["y"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 22)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
y_train = np.array(y_train)  
y_test = np.array(y_test)

X_train = X_train.transpose(0, 2, 3, 1)  # Convert (num_samples, 2, 6, 7) to (num_samples, 6, 7, 2)
X_test = X_test.transpose(0, 2, 3, 1)


In [6]:
def preprocess_board(board):
    if board["whose_turn"] != "red":
        # Flip the layers to make each board look like it's from the perspective of "red", aka plus
        board["x"] = board["x"][::-1, :, :]
    return board

In [7]:
df = df.apply(preprocess_board, axis=1)

In [19]:
cnn = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=128,kernel_size=(4,4),activation=tf.nn.relu,input_shape=(6, 7, 2)),
        # had to adjust data_format because it was treating the images as 2x6 with 7 channels
        tf.keras.layers.MaxPooling2D(pool_size = (2,2),strides=(1, 1), padding = "same"),
        #tf.keras.layers.Conv2D(64, (1, 1), activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.0005)),
        tf.keras.layers.Dense(64,activation=tf.nn.relu,kernel_regularizer = tf.keras.regularizers.l2(0.0005)),
        tf.keras.layers.Dense(7,activation=tf.nn.softmax)
        ])

In [20]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.summary()

cnn.fit(X_train,y_train,epochs=10,validation_split=0.2,batch_size=100)

pred_probs = cnn.predict(X_test)
pred = np.argmax(pred_probs, axis=1)
print(np.mean(pred==y_test))

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 3, 4, 128)      │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 3, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │        98,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 107,207 (418.78 KB)

 Trainable params: 107,207 (418.78 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.3024 - loss: 1.7817 - val_accuracy: 0.4264 - val_loss: 1.5098
Epoch 2/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.4443 - loss: 1.4715 - val_accuracy: 0.4445 - val_loss: 1.4596
Epoch 3/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.4698 - loss: 1.4117 - val_accuracy: 0.4714 - val_loss: 1.4015
Epoch 4/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4828 - loss: 1.3767 - val_accuracy: 0.4753 - val_loss: 1.3871
Epoch 5/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4951 - loss: 1.3503 - val_accuracy: 0.4806 - val_loss: 1.3830
Epoch 6/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4986 - loss: 1.3344 - val_accuracy: 0.4853 - val_loss: 1.3656
Epoch 7/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.5107 - loss: 1.3165 - val_accuracy: 0.4830 - val_loss: 1.3813
Epoch 8/10
1255/1255 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.5180 - loss: 1.2954 - 

In [14]:
print(f"X_train dtype: {X_train.dtype}, shape: {X_train.shape}")


X_train dtype: float64, shape: (156828, 6, 7, 2)
